In [27]:
import numpy as np
import gzip, pickle
import pandas as pd

patients = pd.read_csv('internacoes_charlson_zero.csv.gz', compression='gzip', \
                       nrows=None, usecols=['target'])
target = patients['target'].values
patients.shape

(48907, 1)

In [28]:
with gzip.open("data_5k_unigram.npy.gz", "rb") as wfp:   #Pickling
    unigram = pickle.load(wfp)
    wfp.close()
    
with gzip.open("data_5k_bigram.npy.gz", "rb") as wfp:   #Pickling
    bigram = pickle.load(wfp)
    wfp.close()
    
with gzip.open("data_5k_trigram.npy.gz", "rb") as wfp:   #Pickling
    trigram = pickle.load(wfp)
    wfp.close()
    
with gzip.open("data_10k_multigram.npy.gz", "rb") as wfp:   #Pickling
    multigram = pickle.load(wfp)
    wfp.close()
        
print (unigram.shape, bigram.shape, trigram.shape, multigram.shape)

(48907, 5000) (48907, 5000) (48907, 5000) (48907, 10000)


In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [30]:
from collections import OrderedDict
grams = OrderedDict({ 
    'unigram': unigram,
    'bigram': bigram,
    'trigram': trigram,
    'multigram': multigram,
})

In [31]:
for size in range(250,501,250):
    
    results = {}
    
    for name, gram in grams.items():
        
        model = RandomForestRegressor(n_jobs=5)

        target1 = np.asarray(target[:5000])
        data = gram[:len(target1),:250].todense()

        scores = cross_val_score(model, data, target1, cv=6, scoring='neg_mean_absolute_error')
        results[name] = np.abs(round(np.mean(scores),2))
    
    print(size, results['unigram'], results['bigram'], results['trigram'], results['multigram'])

250 0.97 1.3 1.39 1.34
500 0.97 1.3 1.35 1.34
